# Drop SENTIMP and highly correlated values


Revathi's tasks:
redo yohannes EDA with our csv
do 2 different logistic regressions on ERIOD and SENTIMP as targets
research on time periods
1 = Koon Period (June 13, 1996 through April 30, 2003)
2 = PROTECT Act Period (May 1, 2003 through June 24, 2004)
3 = Booker Period (January 12, 2005 through December 10, 2007)
4 = Gall Period (December 11, 2007 through September 30, 2011)


In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
import pickle

In [2]:
#reading in file 
booker = pd.read_csv('../data/eda_booker_report.csv')
booker = booker.iloc[:, 1:]
booker

,ACCAP,AGE,GGDUM,BOOKER2,AROFFAP,CIRCDIST,MITDUM,MONCIRC,ONSEX,NEWCIT,...,SAFEVALVE,SENTIMP,OTCHPTS,WEAPON,XCRHISSR,LMIN,FY,USSCIDN,ENSPLT0,GDL
0,0,28,0,2,0,80.0,0,10.0,0,1,...,2,1,5,0,3,10,2005,890751.0,8.0,0
1,0,24,0,2,0,80.0,0,10.0,0,0,...,2,1,3,0,2,168,2005,890752.0,72.0,1
2,0,32,0,0,0,33.0,0,5.0,1,0,...,0,1,3,0,2,12,2005,890755.0,15.0,2
3,0,24,1,0,0,33.0,0,5.0,0,0,...,0,1,10,1,5,228,2005,890756.0,270.0,2
4,0,43,0,0,0,93.0,0,11.0,0,1,...,2,1,9,0,4,57,2005,890757.0,60.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,0,40,0,2,0,47.0,0,7.0,1,1,...,2,4,0,0,1,6,2004,759721.0,0.0,4
799996,0,41,0,0,0,47.0,0,7.0,0,1,...,2,1,0,0,1,24,2004,759722.0,24.0,51
799997,0,23,0,0,0,47.0,0,7.0,0,0,...,2,3,1,0,1,6,2004,759723.0,6.0,8
799998,0,30,0,0,0,47.0,0,7.0,0,1,...,2,1,8,0,4,57,2004,759724.0,61.0,0


In [3]:
booker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 28 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ACCAP      800000 non-null  int64  
 1   AGE        800000 non-null  int64  
 2   GGDUM      800000 non-null  int64  
 3   BOOKER2    800000 non-null  int64  
 4   AROFFAP    800000 non-null  int64  
 5   CIRCDIST   800000 non-null  float64
 6   MITDUM     800000 non-null  int64  
 7   MONCIRC    800000 non-null  float64
 8   ONSEX      800000 non-null  int64  
 9   NEWCIT     800000 non-null  int64  
 10  NEWCNVTN   800000 non-null  int64  
 11  EWEDUC     800000 non-null  int64  
 12  NEWRACE    800000 non-null  int64  
 13  OFFTYPE2   800000 non-null  int64  
 14  ERIOD      800000 non-null  int64  
 15  PRIMARY    800000 non-null  int64  
 16  QUARTER    800000 non-null  float64
 17  A          800000 non-null  int64  
 18  SAFEVALVE  800000 non-null  int64  
 19  SENTIMP    800000 non-n

In [4]:
# defining X and y and splitting into traintest sets
y = booker['CIRCDIST']
X = booker.drop(columns = ['CIRCDIST'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

NOTE: ENSPLT0 feature was not included in training because of its high correlation to the target variable, SENTIMP. ENSPLT0 indicates the length of the individuals' sentence, so it had high determination for SENTIMP targets 1, 2 and 3.

In [5]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Logistic Regression

In [6]:
lg = LogisticRegression(random_state = 42)
model_lg = lg.fit(X_train, y_train)

/Users/kaitcrawford/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
print(f"Train Score: {lg.score(X_train,y_train)}")
print(f"Test Score: {lg.score(X_test,y_test)}")

Train Score: 0.4087410714285714
Test Score: 0.40712916666666665


## BayesSearchCV using Logistic Regression

In [8]:
lg = LogisticRegression(random_state =42)


pipe_lg_params = {
    'C': np.logspace(-4, 4, 50), 
    'max_iter': [10_000, 1_000_000],
}   

bs_lg = BayesSearchCV(
    estimator=lg,
    search_spaces=pipe_lg_params,
    n_iter=50, 
    verbose=1,
    cv=5,
    n_jobs=-1
)

In [9]:
bs_lg.fit(X_train,y_train)

AttributeError: module 'numpy' has no attribute 'int'

In [ ]:
# Bayes Search
print(f"Train Score: {bs_lg.score(X_train,y_train)}")
print(f"Test Score: {bs_lg.score(X_test,y_test)}")

In [ ]:
y_pred = bs_lg.predict(X_test)

## GridSearchCV using Logistic Regression

In [ ]:
parameters = [{'penalty':['l1','l2']}, 
              {'C':[1, 10, 100, 1000]}]
gs = GridSearchCV(estimator = lg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)

In [ ]:
gs.fit(X_train, y_train) 

In [ ]:
# Bayes Search
print(f"Train Score: {gs.score(X_train,y_train)}")
print(f"Test Score: {gs.score(X_test,y_test)}")

## Random Forest Classifier  

In [ ]:
# instantiate and fit model
rf = RandomForestClassifier(n_estimators=200, 
                            max_depth=None, 
                            max_features='log2', 
                            min_samples_split=50)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
# Bayes Search
print(f"Train Score: {gs.score(X_train,y_train)}")
print(f"Test Score: {gs.score(X_test,y_test)}")

## Export Models

In [ ]:
with open('../models/circdist/circdist_bs_lg', 'wb') as pickle_out:
    pickle.dump(bs_lg, pickle_out)

with open('../models/circdist/circdist_gs_lg', 'wb') as pickle_out:
    pickle.dump(gs, pickle_out)

with open('../models/circdist/circdist_rf', 'wb') as pickle_out:
    pickle.dump(rf, pickle_out)